# Experiment: Which drugs are "similar", from prescription data?

In the USA the social program for prescription drugs is the "Medicare Part D Prescription Drug" program. Data regarding the prescribed drugs per provider and drug are available at https://data.cms.gov/provider-summary-by-type-of-service/medicare-part-d-prescribers/medicare-part-d-prescribers-by-provider-and-drug/

On this website you can select the columns you want. I downloaded only the code for the drug provider (`Prscrbr_NPI`), the type of provider (`Prscrbr_Type`), the generic name for the drug (`Gnrc_Name`) and the total number of claims made by this provider (`Tot_Clms`):

<center><img src='./image.png' width=50%></img></center>

As a result, I got a 197MB zip file. I made a folder named `data` and placed the file there. I also wrote a utility function to load this data and placed this code in a separate `.py` file (just so I don't pollute this notebook too much!). Let's load the data:

In [1]:
from pathlib import Path

import pandas as pd

from utils import DATA_DIR, load_data

In [ ]:
df = load_data()

## Exploratory analysis

What does the data look like?

In [ ]:
df.shape

- About 25 million rows, 4 columns

In [ ]:
df.info()

- The columns are:

| Column Name  | Description                  | Dtype   | Type meaning |
|--------------|------------------------------|---------|--------------|
| Prscrbr_NPI  | Doctor's ID                  | int64   | an integer   |  
| Prscrbr_Type | Doctor type                  | object  | a string     | 
| Gnrc_Name    | Drug name                    | object  | a string     | 
| Tot_Clms     | Number of prescriptions made | int64   | an integer   | 

In [ ]:
df.head(10)

How many absent items (NaNs) are there in the dataset?

In [ ]:
df.isna().sum()

So there are only five absent items in the `Prscrbr_Type` column. Let's discard the corresponding rows:

In [7]:
df = df.dropna()

In [ ]:
df.shape

The `Prscrbr_NPI`, `Prscrbr_Type`, and `Gnrc_Name` contain categorical values. How many categories in each of them?

In [ ]:
df[['Prscrbr_NPI', 'Prscrbr_Type', 'Gnrc_Name']].nunique()

If we were to create a matrix of `Prscrbr_NPI` by `Gnrc_Name`, with `Tot_Clms` for cell values, that matrix would be of size $1057564 \times 1757$! How many values a matrix of this size would store?

In [ ]:
num_positions = 1057564 * 1757
print(f'{num_positions:,}')

Almost two billion positions! And most of them are zeros, because we only have $25,869,516$ non-zero values.

In [ ]:
num_nonzero_positions = df.shape[0]
print(f'{num_nonzero_positions:,}')

The occupancy ratio of this matrix is very, very small:

In [ ]:
occupancy_ratio = num_nonzero_positions / num_positions
print(f'{occupancy_ratio:.2%}')

Only about $1.39\%$ of the matrix is non-zero. Such matrices are called *sparse matrices*. There are algorithmic techniques to handle sparse matrices in an efficient manner, both in terms of amount of computation to be performed and in memory usage. We will make use of it in a bit.

But we should also consider whether all of these categories in `Prscrbr_NPI` and `Gnrc_Name` are truly useful. Maybe categories with under-representation could be discarded, since we want to focus on the most present information only.

Let's start with the physician type:

In [13]:
count_Prscrbr_Type = df['Prscrbr_Type'].value_counts()

In [ ]:
count_Prscrbr_Type.head(10)

There is a steep decline in representation from the most common type of doctor ("Family Practice") to the 10th most common ("Ophtalmology").

In [ ]:
count_Prscrbr_Type.tail(10)

The least represented types of doctor have only one entry each.

Let's decide on a cutoff value to filter the least common types. It will be quite arbitrary, and it is worthwhile to consider whether this removal of information will qualitatively impact the conclusions to be obtained from the analysis of this dataset - keep this in mind, ok?

In [16]:
import numpy as np

In [ ]:
count_Prscrbr_Type.quantile(np.linspace(0, 1, 21))

Making a list of quantiles from zero to $100\%$, in steps of $5\%$, we see an almost exponential growth in number of physicians per type. Let's cut aggressively then: it will not change the number of data points substantially.

In [18]:
q_Prscrbr_Type = 0.70
cut_value = count_Prscrbr_Type.quantile(q_Prscrbr_Type)

useful_Prscrbr_Type = count_Prscrbr_Type[count_Prscrbr_Type >= cut_value].index

In [ ]:
useful_Prscrbr_Type

In [20]:
df = df[df['Prscrbr_Type'].isin(useful_Prscrbr_Type)]

In [ ]:
df.shape

We only reduced the number of data points from $25,869,516$ to $25,754,507$!

Now let's look into the physicians:

In [22]:
count_Prscrbr_NPI = df['Prscrbr_NPI'].value_counts()

In [ ]:
count_Prscrbr_NPI.quantile(np.linspace(0, 1, 21))

Again, a very exponential-like growth of number of prescriptions per physician. Let's cut aggressively again, but remember: this may impact the quality of the subsequent analysis, so it may be interesting to revisit these arbitrary decisions later.

In [24]:
q_Prscrbr_NPI = 0.5
cut_value = count_Prscrbr_NPI.quantile(q_Prscrbr_NPI)

useful_Prscrbr_NPI = count_Prscrbr_NPI[count_Prscrbr_NPI >= cut_value].index

In [ ]:
len(useful_Prscrbr_NPI)

In [26]:
df = df[df['Prscrbr_NPI'].isin(useful_Prscrbr_NPI)]

In [ ]:
df.shape

We are keeping a large portion of the information still: $24,297,207$, from the previous number of $25,754,507$ rows.

Now let's investigate the drugs:

In [28]:
count_Gnrc_Name = df['Gnrc_Name'].value_counts()

In [ ]:
count_Gnrc_Name.head(10)

In [ ]:
count_Gnrc_Name.tail(10)

In [ ]:
count_Gnrc_Name.quantile(np.linspace(0, 1, 21))

Again, an exponential growth in quantiles, let's cut:

In [32]:
q_Gnrc_Name = 0.5
cut_value = count_Gnrc_Name.quantile(q_Gnrc_Name)

useful_Gnrc_Name = count_Gnrc_Name[count_Gnrc_Name >= cut_value].index

In [ ]:
useful_Gnrc_Name

In [34]:
df = df[df['Gnrc_Name'].isin(useful_Gnrc_Name)]

In [ ]:
df.shape

Finally, after all this filtering, we went from $25,869,521$ items to $24,246,835$.

In [ ]:
print(f'We dropped {1.0 - 24246835/25869521:.2%} of the data')

In [ ]:
num_doctors = df['Prscrbr_NPI'].nunique()
print(f'There are {num_doctors:,} doctors left.')

In [ ]:
num_drugs = df['Gnrc_Name'].nunique()
print(f'There are {num_drugs:,} drugs left.')

In [ ]:
print( \
    f'There are {num_doctors:,} times {num_drugs:,} ' + \
    f'= {num_doctors * num_drugs:,} possible pairs (doctor, drug), ' + \
    'but the vast majority of them will have zero prescriptions.' \
)

In [ ]:
print( \
    f'There are {df.shape[0]:,} records left.' + \
    f' The occupancy ratio is {df.shape[0] / (num_doctors * num_drugs):.2%}.' \
)

Our matrix of prescriptions by doctors and drugs would still be sparse, but with a better occupancy ratio now.

Lets save the results of our exploratory analysis!

In [44]:
df.to_parquet(DATA_DIR / 'cleaned_data.parquet')